# Testing Agents in AI Foundry
This notebook creates and runs an agent from AI Foundry

Best way to set it up is with [uv](https://docs.astral.sh/uv/)

1. in `agents` directory run `uv sync` (if python is missing, install it using `uv python install`)
2. in VSCode press [Ctrl+Shift+P] and select `Python: Select Interpreter`, choose the one from `agents` directory: `./agents/.venv/bin/python3.xx`

In [ ]:
# Install required packages

! pip install requests "semantic-kernel[azure]>=1.35.2" jsonref azure-identity

## Step 2
Create Project client

In [ ]:
from datetime import date
import os
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent
from semantic_kernel.agents import (
    AzureAIAgent,
    AzureAIAgentThread,
    AzureAIAgentSettings,
)
from azure.identity import DefaultAzureCredential, AzureDeveloperCliCredential
from azure.ai.agents.models import (
    OpenApiTool,
    OpenApiAnonymousAuthDetails,
    McpTool,
    ToolDefinition,
)
import jsonref
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv(override=True)

endpoint = os.environ.get("AZURE_AI_FOUNDRY_CONNECTION_STRING")
deployment_name = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", None)
tenant_id = os.environ.get("AZURE_TENANT_ID", None)

ai_agent_settings = AzureAIAgentSettings(
    endpoint=endpoint,
    model_deployment_name=deployment_name,
    api_version=api_version,
)
print(ai_agent_settings)

creds = (
    AzureDeveloperCliCredential(tenant_id=tenant_id)
    if os.environ.get("USE_AZURE_DEV_CLI") == "true"
    else DefaultAzureCredential()
)
# uncomment to test token aquisition
# token_test  = creds.get_token("https://ai.azure.com")
# print(f"Token for https://ai.azure.com: {token_test.token[:10]}...")

client = AzureAIAgent.create_client(
    credential=creds,
    endpoint=ai_agent_settings.endpoint,
    api_version=ai_agent_settings.api_version,
)

# List agents
print("\n --- Agents ---")
async for agent in client.agents.list_agents():
    print(
        f"Agent ID: {agent.id}, Name: {agent.name}, Description: {agent.description}, Deployment Name: {agent.model}"
    )

print("\n --- Connections ---")
# List connections
async for connection in client.connections.list():
    print(
        f"Connection ID: {connection.id}, Name: {connection.name}, Type: {connection.type} Default: {connection.is_default}"
    )

### Prepare Tools

In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", None)
openapi_server_url = os.environ.get("OPENAPI_SERVER_URL", None)
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "tool")

mcp_tools: list[ToolDefinition] = []
openapi_tools: list[ToolDefinition] = []
mcp_tool: McpTool | None = None

if mcp_server_url:
    mcp_tool = McpTool(
        server_label=mcp_server_label,
        server_url=mcp_server_url,
        allowed_tools=[],  # Optional: specify allowed tools
    )
    mcp_tool.set_approval_mode("never")

    print(
        f"Using MCP Tool with server URL: {mcp_server_url} and label: {mcp_server_label}"
    )
    mcp_tools = mcp_tools + mcp_tool.definitions

    # add same tool again with different label
    mcp_tool_2 = McpTool(
        server_label="current_weather",
        server_url=mcp_server_url,
        allowed_tools=[],  # Optional: specify allowed tools
    )
    mcp_tool_2.set_approval_mode("never")
    mcp_tools = mcp_tools + mcp_tool_2.definitions


if openapi_server_url:
    with open("weather.json", "r") as f:
        openapi_weather = jsonref.loads(f.read())
        openapi_weather["servers"] = [{"url": openapi_server_url}]

    openapi_tool = OpenApiTool(
        name="WeatherAPI",
        spec=openapi_weather,
        auth=OpenApiAnonymousAuthDetails(),
        description="Retrieve weather information for a location",
        # allowed_tools=[],  # Optional: specify allowed tools
    )
    print(f"Using OpenAPI Tool with server URL: {openapi_server_url}")
    openapi_tools = openapi_tools + openapi_tool.definitions

### Create Agents

In [ ]:
user_message = (
    "You are a reliable, funny and amusing weather forecaster named Jonny Weather. "
    "You provide weather forecasts in a humorous and engaging manner. "
    "You like to use puns and jokes to make the weather more entertaining. "
    "You love to use emojis to make your forecasts more colorful and fun. "
    "You are very knowledgeable about the weather, history of forecasting, and the science behind it. "
    "You've been doing this for a long time and have a lot of experience. Love to talk about your adventures in the field."
    "You always use the weather tools provided to you, never make stuff up. "
    "You've been living in Galway, Ireland for the past 20 years, you've seen it all and done it all. ☔️🌈"
    "\n"
    "## Rules\n"
    "* Always provide a weather forecast for today and tomorrow."
    "* Use emojis to make your forecasts more colorful and fun."
    "* Use tables to present weather."
    "* If you don't know the answer, say 'I don't know' or 'I don't have that information'."
)
# agent_instructions = "You're a helpful agent"


async def create_agent(
    agent_name: str, agent_instructions: str, tools: list[ToolDefinition]
) -> AzureAIAgent:

    agent_definition = None
    async for agent in client.agents.list_agents():
        if agent.name == agent_name and agent_definition is None:
            agent_definition = agent
            break

    if agent_definition:
        print(
            f"Found existing agent with ID: {agent_definition.id} and name: {agent_definition.name}"
        )
        agent_definition = await client.agents.update_agent(
            agent_id=agent_definition.id,
            instructions=agent_instructions,
            model=ai_agent_settings.model_deployment_name,
            tools=tools,
            temperature=0.2,
        )
        print(
            f"Updated agent with id {agent_definition.id} name: {agent_name} with model {ai_agent_settings.model_deployment_name}"
        )
    else:
        agent_definition = await client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name=agent_name,
            instructions=agent_instructions,
            tools=tools,
            temperature=0.2,
        )
        print(
            f"Created agent with id {agent_definition.id} name: {agent_name} with model {ai_agent_settings.model_deployment_name}"
        )

    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
    )
    return agent


agent_name_mcp = "Jonny_Weather_mcp"
agent_name_openapi = "Jonny_Weather_openapi"
agent_name_logicapp = "Jonny_Weather_logicapp"

mcp_agent = await create_agent(
    agent_name=agent_name_mcp, agent_instructions=user_message, tools=mcp_tools
)

openapi_agent = await create_agent(
    agent_name=agent_name_openapi, agent_instructions=user_message, tools=openapi_tools
)

### Use the agent

In [ ]:
user_message = "What's the weather in NYC"


async def on_intermediate_message(agent_response: ChatMessageContent):
    print(f"Intermediate response from MCP Agent: {agent_response}")
    for item in agent_response.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"Function Result:> {item.result} for function: {item.name}")
        elif isinstance(item, FunctionCallContent):
            print(f"Function Call:> {item.name} with arguments: {item.arguments}")
        else:
            print(f"{item}")


async def test_mcp_foundry_agent():
    print("--------------------- Invoking MCP Agent using AI FOUNDRY SDK")
    try:
        thread = await client.agents.threads.create()
        message = await client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_message,
        )
        run = await client.agents.runs.create_and_process(
            thread_id=thread.id,
            agent_id=mcp_agent.id,
            tool_resources=mcp_tool.resources if mcp_tool else None,
        )

        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Retrieve the steps taken during the run for analysis
        run_steps = client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)

        # Loop through each step to display information
        async for step in run_steps:
            print(f"Step {step['id']} status: {step['status']}")

            # Fetch and log all messages
            messages = client.agents.messages.list(thread_id=thread.id)
            async for message in messages:
                print(f"Role: {message.role}, Content: {message.content}")

            tool_calls = step.get("step_details", {}).get("tool_calls", [])
            for call in tool_calls:
                print(f"  Tool Call ID: {call.get('id')}")
                print(
                    f"  Type: {call.get('type')} - server_label: {call.get('server_label')} - name: {call.get('name')} with output: {call.get('output')}"
                )
                function_details = call.get("function", {})
                if function_details:
                    print(f"  Function name: {function_details.get('name')}")
                    print(f" function output: {function_details.get('output')}")

            print()

    except Exception as e:
        print(f"Error invoking MCP Agent: {e}")


async def test_mcp_sk_agent():
    print(
        f"--------------------- Invoking MCP Agent {mcp_agent.id} using Semantic Kernel SDK"
    )
    try:
        if mcp_tool and mcp_agent:
            mcp_tool.set_approval_mode("never")
            thread = AzureAIAgentThread(client=client)
            async for agent_response in mcp_agent.invoke(
                messages=user_message,
                thread=thread,
                additional_instructions="Today is " + date.today().strftime("%Y-%m-%d"),
                tools=mcp_tool.resources,
                on_intermediate_message=on_intermediate_message,
            ):
                print(f"MCP Agent: {agent_response}")
                thread = agent_response.thread
        else:
            print("MCP Agent or tool not available")
    except Exception as e:
        print(f"Error invoking MCP Agent: {e}")


async def test_openapi_agent():
    print("--------------------- Invoking OPEN API Agent using Semantic Kernel SDK")
    try:
        if openapi_agent:
            thread = AzureAIAgentThread(client=client)
            async for agent_response in openapi_agent.invoke(
                messages=user_message,
                thread=thread,
                additional_instructions="Today is " + date.today().strftime("%Y-%m-%d"),
            ):
                print(f"OpenAPI Agent: {agent_response}")
        else:
            print("OpenAPI Agent not available")
    except Exception as e:
        print(f"Error invoking OpenAPI Agent: {e}")


# Run the async function
# await test_mcp_foundry_agent()
await test_mcp_sk_agent()
# await test_openapi_agent()

In [ ]:
codex_tool = McpTool(
    server_label="codex",
    server_url="https://gitmcp.io/openai/codex",
)
codex_tool.set_approval_mode("never")
codex_tools: list[ToolDefinition] = [] + codex_tool.definitions

codex_agent = await create_agent(
    agent_name="Codex-Agent",
    agent_instructions="you are a helpful assistant that can answer questions about the codebase",
    tools=codex_tools,
)

codex_thread = AzureAIAgentThread(client=client)
async for agent_response in codex_agent.invoke(
    messages="summarize the code in the repository I pointed the MCP tool to",
    thread=codex_thread,
    additional_instructions="Today is " + date.today().strftime("%Y-%m-%d"),
    tools=codex_tool.resources,
    on_intermediate_message=on_intermediate_message,
):
    print(f"Codex Agent: {agent_response}")
    codex_thread = agent_response.thread